In [1]:
import pandas as pd
import numpy as np
import sys
import config as cfg
sys.path.append("./tools/")
import funclib
import exact_ec_from_uniprot as exactec
from sklearn import metrics

## Case 1: Ecoli

In [2]:
ground_truth=pd.read_csv('/home/shizhenkun/codebase/DMLF/data/ecoli_filtered.tsv', sep='\t').iloc[:,1:]
ground_truth['ec_number'] = ground_truth.ec_number.apply(lambda x: str(x).replace(';', ', '))

res_deepec = pd.read_csv('/home/shizhenkun/codebase/DMLF/results/deepec_ecoli/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number'], header=0 )
res_deepec.ec_number=res_deepec.apply(lambda x: x['ec_number'].replace('EC:',''), axis=1)
res_deepec.columns = ['id','ec_deepec']
res_deepec['isemzyme_deepec']=res_deepec.ec_deepec.apply(lambda x: True if str(x)!='nan' else False)
res_deepec['functionCounts_deepec'] = res_deepec.ec_deepec.apply(lambda x :len(str(x).split(',')))
res_deepec = res_deepec.groupby(['id'], sort=False)['ec_deepec'].apply(lambda x:x.str.cat(sep=', ')).reset_index()

res_big = ground_truth.merge(res_deepec, on='id', how='left')

In [3]:
ground_truth[ground_truth.ec_number!='nan']

,id,gene_name,protein_name,ec_number,seq
1,b0002,thrA,Bifunctional aspartokinase/homoserine dehydrog...,"1.1.1.3, 2.7.2.4",MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...
2,b0003,thrB,homoserine kinase,2.7.1.39,MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...
3,b0004,thrC,L-threonine synthase,4.2.3.1,MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...
7,b0008,talB,transaldolase B,2.2.1.2,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...
8,b0009,mog,molybdochelatase incorporating molybdenum into...,2.7.7.75,MNTLRIGLVSISDRASSGVYQDKGIPALEEWLTSALTTPFELETRL...
...,...,...,...,...,...
4191,b4539,yoeB,toxin of the YoeB-YefM toxin-antitoxin system,3.1.-.-,MKLIWSEESWDDYLYWQETDKRIVKKINELIKDTRRTPFEGKGKPE...
4211,b4565,sgcB,putative enzyme IIB component of PTS,2.7.1.69,MKKILVACGTGMSTSTMIAHKLQEFLTEQGISATTAQCCLNEIPLN...
4217,b4571,wbbL,"pseudogene, lipopolysaccharide biosynthesis pr...",2.4.1.-,MVYIIIVSHGHEDYIKKLLENLNADDEHYKIIVRDNKDSLLLKQIC...
4222,b4579,yaiX,"pseudogene, interrupted by IS2A; hexapeptide t...",2.3.1.-,MDLLPFLLDANLSATNPPAIPHWWKRQPLIPNLLSQELKNYLKLNV...


In [4]:
res_ecrecer = pd.read_csv('/home/shizhenkun/codebase/DMLF/tmp/ecolip.tsv', sep='\t').iloc[:,np.r_[0,1,4:9]].rename(columns={'ec_number':'ec_ecrecer'})
res_ecrecer['ec_ecrecer'] = res_ecrecer.ec_ecrecer.apply(lambda x: str(x).replace(',', ', '))

res_big = res_big.merge(res_ecrecer, on='id', how='left')
res_big.loc[res_big[res_big.ec_number=='nan'].index.values,'ec_number']='-'
res_big.loc[res_big[res_big.ec_number.isnull()].index.values,'ec_number']='-'
res_big.loc[res_big[res_big.ec_deepec.isnull()].index.values,'ec_deepec']='-'

In [5]:
res_big['correct_deepec']=res_big.apply(lambda x: True if (set([i.strip(' ') for i in set(x.ec_number.split(','))]) == set([i.strip(' ') for i in set(x.ec_deepec.split(','))])) else False, axis=1)
res_big['correct_ecrecer']=res_big.apply(lambda x: True if (set([i.strip(' ') for i in set(x.ec_number.split(','))]) == set([i.strip(' ') for i in set(x.ec_ecrecer.split(','))])) else False, axis=1)
res_big['function_counts']=res_big.ec_deepec.apply(lambda x:len(str(x).split(',')))

res_big.loc[res_big[res_big.correct_deepec].index.values,'ec_deepec']=res_big[res_big.correct_deepec].ec_number.values
res_big.loc[res_big[res_big.correct_ecrecer].index.values,'ec_ecrecer']=res_big[res_big.correct_ecrecer].ec_number.values

In [85]:
res_big.to_excel('/home/shizhenkun/codebase/DMLF/results/ecolicase_results.xlsx', index=None)

In [6]:
total = res_big.shape[0]
correct_deepec = len(res_big[res_big.correct_deepec])
correct_ecrecer = len(res_big[res_big.correct_ecrecer])

print('Accuracy of deepec: {0}({1}/{2})'.format(correct_deepec/total, correct_deepec, total))
print('Accuracy of ecrecer: {0}({1}/{2})'.format(correct_ecrecer/total, correct_ecrecer, total))

Accuracy of deepec: 0.7947709393799167(3435/4322)
Accuracy of ecrecer: 0.8519204072188802(3682/4322)


In [7]:
def cac_metrics(res_big):

    ground_ec = [str(i).split(',') for i in list(res_big.ec_number.values)]   #groud_truth ec numbers
    ground_ec_set = set([i.strip(' ') for item in ground_ec for i in item])

    deep_ec = [str(i).split(',') for i in list(res_big.ec_deepec.values)]    #deepec pred ec numbers
    deep_ec_set = set([i.strip(' ') for item in ground_ec for i in item])

    deep_ec = [str(i).split(',') for i in list(res_big.ec_deepec.values)]    #ec_ecrecer pred ec numbers
    deep_ec_set = set([i.strip(' ') for item in ground_ec for i in item])

    final_ec_set = ground_ec_set | deep_ec_set | deep_ec_set
    label_dict = dict(zip(final_ec_set,range(len(final_ec_set))))

    ground_ec_set = set(res_big.ec_number.values)
    deep_ec_set = set(res_big.ec_deepec.values)
    ecrecer_ec_set = set(res_big.ec_ecrecer.values)

    final_ec_set = ground_ec_set | ecrecer_ec_set | deep_ec_set
    label_dict = dict(zip(final_ec_set,range(len(final_ec_set))))

    ground_label = res_big.ec_number.apply(lambda x: label_dict.get(x)).values
    deepec_label = res_big.ec_deepec.apply(lambda x: label_dict.get(x)).values
    ecrecer_label = res_big.ec_ecrecer.apply(lambda x: label_dict.get(x)).values
    
    acc_deepec = metrics.accuracy_score(ground_label, deepec_label)
    acc_ecrecer = metrics.accuracy_score(ground_label, ecrecer_label)
    
    precision_deepec = metrics.precision_score(ground_label, deepec_label, average='macro', zero_division=True )
    precision_ecrecer = metrics.precision_score(ground_label, ecrecer_label, average='macro', zero_division=True )

    recall_deepec = metrics.recall_score(ground_label, deepec_label, average='macro', zero_division=True)
    recall_ecrecer = metrics.recall_score(ground_label, ecrecer_label, average='macro', zero_division=True)

    f1_deepec = metrics.f1_score(ground_label, deepec_label, average='macro', zero_division=True)
    f1_ecrecer= metrics.f1_score(ground_label, ecrecer_label, average='macro', zero_division=True)

    print('%12s'%'baslineName', '\t\t', 'accuracy','\t', 'precision-macro \t', 'recall-macro','\t', 'f1-macro')
    print('%12s'%'deepec', ' \t\t%f '%acc_deepec,'\t%f'% precision_deepec, '\t\t%f'% recall_deepec,'\t%f'% f1_deepec)
    print('%12s'%'ec_recer', ' \t\t%f '%acc_ecrecer,'\t%f'% precision_ecrecer, '\t\t%f'% recall_ecrecer,'\t%f'% f1_ecrecer)


In [8]:
cac_metrics(res_big)

 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
      deepec  		0.794771  	0.762929 		0.738938 	0.526728
    ec_recer  		0.851920  	0.689161 		0.826361 	0.542161


In [12]:
res_big[res_big.res_type!='db_match']

,id,gene_name,protein_name,ec_number,seq,ec_deepec,id_uniprot,ec_ecrecer,date_integraged,date_sequence_update,date_annotation_update,res_type,correct_deepec,correct_ecrecer,function_counts
202,b0218,yafU,pseudogene,-,RTIEFIMSSERDLVNFLGDFSMDVAKAVIAGGVATAIGSLASFACV...,-,P77354,-,1997-11-01,1997-02-01,2020-10-07,blast_match,True,True,1
213,b0229,lfhA,"pseudogene, flagellar system protein, promoter...",-,LLTINFIVVTKGAERISEVSARFTLDAMPGKQMAIDADLNAGLINQ...,-,Q47153,-,1997-11-01,1997-11-01,2021-09-29,blast_match,True,True,1
214,b0230,lafU,"pseudogene, lateral flagellar motor protein fr...",-,VSKSERESIIAALHGQSIFNGGGLSPLNKISPSHPPKPATVAVPEE...,-,D3GSC3,-,2015-04-01,2010-03-23,2021-09-29,blast_match,True,True,1
219,b0235,ykfJ,pseudogene,-,MGKYIRPLSDAVFTIASDDLWIESLAIQQLHTTANLPNMQRVVGMP...,-,P75675,-,2002-05-10,1997-02-01,2020-10-07,blast_match,True,True,1
220,b0236,prfH,"pseudogene, RF-1 domain family",-,VLETETGRYSDTLRSALVSLDGDNAWALSESWCGTIQWICPSPYRP...,-,P28369,-,1992-12-01,2006-09-19,2021-09-29,blast_match,True,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4310,b4694,yagP,"pseudogene, LysR family, fragment",-,HKCIGWRQSPEVAPYRWPFEENGRTFDLAIEPQITTNDLRLMLRLA...,-,P75684,-,1997-11-01,1997-02-01,2021-06-02,blast_match,True,True,1
4311,b4695,ykgT,"pseudogene, MobA-related",-,MPAPVVLILAAGRGECFLASGGNTHK,-,P75684,-,1997-11-01,1997-02-01,2021-06-02,blast_match,True,True,1
4312,b4696,yneO,"pseudogene, AidA homolog",-,MRKSSGLTTKFSRLTLGVLLALSGSASGASLEVDNDQITNIDTDVA...,-,P32051,-,1993-10-01,1997-11-01,2021-04-07,blast_match,True,True,1
4313,b4697,yrdF,"pseudogene, conserved protein",-,MKSLFNRLTGKAVSRTAFVEHLGQEVIQHHPNWKVMISTDHKLMRID,-,NaN,-,NaN,NaN,NaN,dmlf_pred,True,True,1


## Case 2: 2021 Data

In [ ]:
# 2020 data
! tar -zxvf ./tmp/uniprot_sprot-only2020_06.tar.gz -C ./tmp/
! mv ./tmp/uniprot_sprot.dat.gz ./tmp/sprot2020.data.gz
! rm -rf ./tmp/*.fasta.gz ./tmp/*.xml.gz

# latest data
! mv $cfg.DATADIR'uniprot_sprot_latest.dat.gz' $cfg.TEMPDIR$currenttime'_uniprot_sprot_latest.dat.gz'
install_axel()
!axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz -o ./data/uniprot_sprot_latest.dat.gz -q -c

In [13]:
exactec.run_exact_task(infile=cfg.TEMPDIR+'sprot2020.data.gz', outfile=cfg.DATADIR+'case_sprot_2020.tsv')

563972it [03:19, 2828.74it/s]

finished use time 198.923 s


In [22]:
sprot_2020 = pd.read_csv(cfg.DATADIR+'case_sprot_2020.tsv', sep='\t')

sprot_2020.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot_2020.reset_index(drop=True, inplace=True)

sprot_latest= pd.read_feather(cfg.DATADIR + 'latest_sprot.feather')

case_data = sprot_latest[~sprot_latest.seq.isin(sprot_2020.seq)].reset_index(drop=True)
case_data = case_data[~case_data.id.isin(sprot_2020.id)].reset_index(drop=True)

In [35]:
funclib.table2fasta(table=case_data[['id','seq']], file_out=cfg.DATADIR+'case_2021.fasta')

Write finished


In [23]:
case_deepec = pd.read_csv('/home/shizhenkun/codebase/DMLF/results/deepec_case2021/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number'], header=0 )
case_deepec.ec_number=case_deepec.apply(lambda x: x['ec_number'].replace('EC:',''), axis=1)
case_deepec.columns = ['id','ec_deepec']
case_deepec['isemzyme_deepec']=case_deepec.ec_deepec.apply(lambda x: True if str(x)!='nan' else False)
case_deepec['functionCounts_deepec'] = case_deepec.ec_deepec.apply(lambda x :len(str(x).split(',')))
case_deepec = case_deepec.groupby(['id'], sort=False)['ec_deepec'].apply(lambda x:x.str.cat(sep=', ')).reset_index()

In [24]:
case_data = case_data.merge(case_deepec, on='id', how='left')

case_ecrecer = pd.read_csv('/home/shizhenkun/codebase/DMLF/results/case_2021_p_o.tsv', sep='\t').iloc[:,np.r_[0,1,4:9]].rename(columns={'ec_number':'ec_ecrecer'})
case_ecrecer['ec_ecrecer'] = case_ecrecer.ec_ecrecer.apply(lambda x: str(x).replace(',', ', '))
case_data = case_data.merge(case_ecrecer.iloc[:,np.r_[0,2,6]], on='id', how='left')

In [25]:
case_data.loc[case_data[case_data.ec_deepec.isnull()].index.values,'ec_deepec']='-'
case_data['correct_deepec']=case_data.apply(lambda x: True if (set([i.strip(' ') for i in set(x.ec_number.split(','))]) == set([i.strip(' ') for i in set(x.ec_deepec.split(','))])) else False, axis=1)
case_data['correct_ecrecer']=case_data.apply(lambda x: True if (set([i.strip(' ') for i in set(x.ec_number.split(','))]) == set([i.strip(' ') for i in set(x.ec_ecrecer.split(','))])) else False, axis=1)
case_data.loc[case_data[case_data.correct_deepec].index.values,'ec_deepec']=case_data[case_data.correct_deepec].ec_number.values
case_data.loc[case_data[case_data.correct_ecrecer].index.values,'ec_ecrecer']=case_data[case_data.correct_ecrecer].ec_number.values

In [26]:
total = case_data.shape[0]
correct_deepec = len(case_data[case_data.correct_deepec])
correct_ecrecer = len(case_data[case_data.correct_ecrecer])

print('Accuracy of deepec: {0}({1}/{2})'.format(correct_deepec/total, correct_deepec, total))
print('Accuracy of ecrecer: {0}({1}/{2})'.format(correct_ecrecer/total, correct_ecrecer, total))

Accuracy of deepec: 0.5208333333333334(1025/1968)
Accuracy of ecrecer: 0.8821138211382114(1736/1968)


In [27]:
case_data['ec7'] = case_data.ec_number.apply(lambda x: True if '7' in x[0] else False)

In [28]:
case_data[(~case_data.ec7)&(case_data.correct_ecrecer)&(~case_data.isenzyme)]

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength,ec_deepec,ec_ecrecer,res_type,correct_deepec,correct_ecrecer,ec7
4,O85043,CSS4A_HALNC,False,False,0,-,0,2021-02-10,1998-11-01,2021-09-29,MKIMQVEKTLVSTNRIADMGHKPLLVVWEKPGAPRQVAVDAIGCIP...,83,-,-,db_match,True,True,False
5,Q18694,UN130_CAEEL,False,False,0,-,0,2021-02-10,1996-11-01,2021-06-02,MLFSMESILSSTKPKLEPPPKLEPEVTINEQVVDLPRSNTRLSEPS...,333,-,-,db_match,True,True,False
8,P0DTF9,PTIP2_HUMAN,False,False,0,-,0,2021-02-10,2021-02-10,2021-09-29,MCWLRAWGQILLPVFLSLFLIQLLISFSENGFIHSPRNNQKPRDGN...,154,-,-,db_match,True,True,False
13,Q38E83,LSM12_TRYB2,False,False,0,-,0,2021-02-10,2005-11-22,2021-04-07,MPVCNNDSQLIGLGVSVTLADDTTVNGTVYTYNSSEGLLVLFQGFS...,190,-,-,db_match,True,True,False
14,Q7V6G4,CSOSE_PROMM,False,False,0,-,0,2021-02-10,2003-10-01,2021-09-29,MPKPSSSSSSDSPSLTTAKPAGSEVAKKVLPAKATQTVDVSASTSS...,204,-,-,db_match,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,G5EF51,KY_CAEEL,False,False,0,-,0,2021-09-29,2011-12-14,2021-09-29,MNSKQHCNRCGKQVYPTDKVGPLKDSTFFHQGCFKCYICGTRLALK...,723,-,-,blast_match,True,True,False
1955,F4K4Y5,DEKP4_ARATH,False,False,0,-,0,2021-09-29,2011-06-28,2021-09-29,MGEEDTKVIVEPTANGTSSLQKTSDAISGKEVQENASGKEVQESKK...,778,-,-,dmlf_pred,True,True,False
1958,A0A0U5GHC1,AUSH_ASPCI,False,False,0,-,0,2021-09-29,2016-03-16,2021-09-29,MPPTREELVSTALNFIAQFAKLDVDSVLSFLSPNCTLRSFPSSLGK...,147,-,-,blast_match,True,True,False
1959,A0A0F7TRU0,AUSH_PENBI,False,False,0,-,0,2021-09-29,2015-07-22,2021-09-29,MSPTRDELLCTALDFVAQFAKLDPESVLSFLSPSCTLRSFPSSLGK...,147,-,-,blast_match,True,True,False


In [29]:
case_data[(~case_data.ec7)&(case_data.correct_deepec)&(~case_data.isenzyme)]

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength,ec_deepec,ec_ecrecer,res_type,correct_deepec,correct_ecrecer,ec7
4,O85043,CSS4A_HALNC,False,False,0,-,0,2021-02-10,1998-11-01,2021-09-29,MKIMQVEKTLVSTNRIADMGHKPLLVVWEKPGAPRQVAVDAIGCIP...,83,-,-,db_match,True,True,False
5,Q18694,UN130_CAEEL,False,False,0,-,0,2021-02-10,1996-11-01,2021-06-02,MLFSMESILSSTKPKLEPPPKLEPEVTINEQVVDLPRSNTRLSEPS...,333,-,-,db_match,True,True,False
8,P0DTF9,PTIP2_HUMAN,False,False,0,-,0,2021-02-10,2021-02-10,2021-09-29,MCWLRAWGQILLPVFLSLFLIQLLISFSENGFIHSPRNNQKPRDGN...,154,-,-,db_match,True,True,False
13,Q38E83,LSM12_TRYB2,False,False,0,-,0,2021-02-10,2005-11-22,2021-04-07,MPVCNNDSQLIGLGVSVTLADDTTVNGTVYTYNSSEGLLVLFQGFS...,190,-,-,db_match,True,True,False
14,Q7V6G4,CSOSE_PROMM,False,False,0,-,0,2021-02-10,2003-10-01,2021-09-29,MPKPSSSSSSDSPSLTTAKPAGSEVAKKVLPAKATQTVDVSASTSS...,204,-,-,db_match,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,G5EF51,KY_CAEEL,False,False,0,-,0,2021-09-29,2011-12-14,2021-09-29,MNSKQHCNRCGKQVYPTDKVGPLKDSTFFHQGCFKCYICGTRLALK...,723,-,-,blast_match,True,True,False
1955,F4K4Y5,DEKP4_ARATH,False,False,0,-,0,2021-09-29,2011-06-28,2021-09-29,MGEEDTKVIVEPTANGTSSLQKTSDAISGKEVQENASGKEVQESKK...,778,-,-,dmlf_pred,True,True,False
1958,A0A0U5GHC1,AUSH_ASPCI,False,False,0,-,0,2021-09-29,2016-03-16,2021-09-29,MPPTREELVSTALNFIAQFAKLDVDSVLSFLSPNCTLRSFPSSLGK...,147,-,-,blast_match,True,True,False
1959,A0A0F7TRU0,AUSH_PENBI,False,False,0,-,0,2021-09-29,2015-07-22,2021-09-29,MSPTRDELLCTALDFVAQFAKLDPESVLSFLSPSCTLRSFPSSLGK...,147,-,-,blast_match,True,True,False


In [32]:
case_data[case_data.isenzyme]

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength,ec_deepec,ec_ecrecer,res_type,correct_deepec,correct_ecrecer,ec7
0,A0A0S2PM82,TCS3_CAMSI,True,False,1,2.1.1.160,4,2021-02-10,2016-02-17,2021-04-07,MELAMGKVNEVLFMNRGEGESSYAQNSSFTQQVASMARPALENAVK...,368,2.1.1.160,2.1.1.160,db_match,True,True,False
1,Q9P4D8,CHMU_PICAN,True,False,1,5.4.99.5,4,2021-02-10,2000-10-01,2021-09-29,MDFMKPETVLDLGNIRDALVRMEDTIIFNFIERSQFYASPSVYKVN...,280,-,5.4.99.5,db_match,False,True,False
2,K9IMD0,TRLF_DESRO,True,False,1,3.4.21.-,3,2021-02-10,2013-02-06,2021-06-02,MKLLFLALLSLLALGPSLAARRRGVRWCTISKPEAAKCSKLQQNLK...,708,-,3.4.21.-,db_match,False,True,False
3,Q31HD6,CSOCA_HYDCU,True,False,1,4.2.1.1,4,2021-02-10,2005-12-06,2021-06-02,MNRLKKSHRQKSLFWRPIAPNPRWQKENPTAHGSTDTGGFGYNGGN...,528,-,4.2.1.1,db_match,False,True,False
6,G5EB37,CHMU_EMENI,True,False,1,5.4.99.5,4,2021-02-10,2011-12-14,2021-09-29,MDTAIDLSDASKALDLANIRFQLIRLEDTITFHLIERVQFPLNKTI...,267,-,5.4.99.5,db_match,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962,A0A0F7TSZ5,AUSI_PENBI,True,False,1,1.-.-.-,1,2021-09-29,2015-07-22,2021-09-29,MLHETISLALLGQPLVPGLMVVSAILYLLYSTQQWRPKNLPLLNDA...,500,-,1.-.-.-,blast_match,False,True,False
1964,S8FIE4,LAC1_FOMPI,True,False,1,1.10.3.2,4,2021-09-29,2013-10-16,2021-09-29,MAFTAISLFLAALGVINTAFAQSAVIGPVTDLDIINAEVNLDGFPR...,539,1.10.3.2,1.10.3.2,blast_match,True,True,False
1965,S8FGV1,LAC2_FOMPI,True,False,1,1.10.3.2,4,2021-09-29,2013-10-16,2021-09-29,MLLSSAFVGSCLAILNFAAAVSAQGGLSRTTLNIVNKVISPDGYSR...,530,1.10.3.2,1.10.3.2,blast_match,True,True,False
1966,Q57286,MMDB_VEIPA,True,False,1,7.2.4.3,4,2021-09-29,1996-11-01,2021-09-29,MEAFAVAIQSVINDSGFLAFTTGNAIMILVGLILLYLAFAREFEPL...,373,4.1.1.70,7.2.4.5,blast_match,False,False,True


In [33]:
cac_metrics(case_data)

 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
      deepec  		0.520833  	0.770489 		0.318440 	0.109649
    ec_recer  		0.882114  	0.766194 		0.817945 	0.604000


## Case 3: 撕裂蜡孔菌 Data

In [3]:
def init_deepec(ecrecods):
    case_deepec = ecrecods
    case_deepec.ec_number=case_deepec.apply(lambda x: x['ec_number'].replace('EC:',''), axis=1)
    case_deepec.columns = ['id','ec_deepec']
    case_deepec = case_deepec.groupby(['id'], sort=False)['ec_deepec'].apply(lambda x:x.str.cat(sep=', ')).reset_index()
    return case_deepec

In [16]:
case_cl_deepec = pd.read_csv('/home/shizhenkun/codebase/DMLF/results/deepec_撕裂蜡孔菌/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number'], header=0 )
case_cl_ecrecer = pd.read_csv('/home/shizhenkun/codebase/DMLF/results/case_撕裂蜡孔菌_p.tsv', sep='\t').rename(columns={'ec_number':'ec_ecrecer'})
case_cl_deepec = init_deepec(case_cl_deepec)

cl_res = case_cl_ecrecer.merge(case_cl_deepec, on='id', how='left')
cl_res.loc[cl_res[cl_res.ec_deepec.isnull()].index.values,'ec_deepec']='-'

## Case 4: 谷棒 Data

In [4]:
case_cl_deepec = pd.read_csv('/home/shizhenkun/codebase/DMLF/results/deepec_deepec_谷棒/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number'], header=0 )
case_cl_ecrecer = pd.read_csv('/home/shizhenkun/codebase/DMLF/results/case_谷棒_p.tsv', sep='\t').rename(columns={'ec_number':'ec_ecrecer'})
case_cl_deepec = init_deepec(case_cl_deepec)

cl_res = case_cl_ecrecer.merge(case_cl_deepec, on='id', how='left')
cl_res.loc[cl_res[cl_res.ec_deepec.isnull()].index.values,'ec_deepec']='-'

In [5]:
cl_res[cl_res.res_type=='blast_match']

,id,id_uniprot,seq,seqlength,ec_ecrecer,date_integraged,date_sequence_update,date_annotation_update,res_type,ec_deepec
0,tr|Q8NPT4|Q8NPT4_CORGL,P96329,MSGLFTPFSDAAKNNTVKTDGDSVSGRDLPITKISEDRFERSAYSA...,NaN,-,2000-12-01,2000-12-01,2021-06-02,blast_match,-
1,tr|Q8NTU3|Q8NTU3_CORGL,Q97CL5,MNTDPAYVAEQTGKLIDAFLTTDPLEPLLDAAKNGVCTEAMGALVT...,NaN,2.8.1.12,2002-02-11,2001-10-01,2021-09-29,blast_match,-
2,tr|Q8NNI4|Q8NNI4_CORGL,P46351,MVVSQGWRNLGVKDYQLERFCCMAVPEFIVSLREKVGQDPLWLPAV...,NaN,-,1995-11-01,1995-11-01,2020-10-07,blast_match,-
3,tr|Q8NT50|Q8NT50_CORGL,P59950,MTELIILLIVIVTALAFDFTNGFHDTGNAMATSIATGALKPKVAVA...,NaN,-,2003-10-31,2003-10-31,2021-06-02,blast_match,-
4,tr|Q79VF1|Q79VF1_CORGL,P63520,MGADQIAAVSGNSAWMLMSASLVLLMTPALALFYGGMSRQKSVLNM...,NaN,-,2004-10-11,2004-10-11,2021-09-29,blast_match,-
...,...,...,...,...,...,...,...,...,...,...
1307,tr|Q8NQX7|Q8NQX7_CORGL,O31833,MKRYAHPRRMSERFPMNPRILMLLRIIFGAAFLALLVLQVLLAIKI...,NaN,-,2009-01-20,1998-01-01,2021-04-07,blast_match,-
1308,tr|Q8NR67|Q8NR67_CORGL,Q9RA45,MKRHHVTQLNTKGVVLQGWDPEDPEHWDSKIAWRTLWITTFSMIIG...,NaN,-,2017-05-10,2012-05-16,2021-09-29,blast_match,-
1309,tr|Q79VF7|Q79VF7_CORGL,P9WGN9,MTDSQTESLSTQSVKPAKKRSWFNSLYTGDGGIDFIAKTKLWYWIT...,NaN,-,2014-04-16,2014-04-16,2021-09-29,blast_match,-
1310,tr|Q8NTH4|Q8NTH4_CORGL,Q5XCG7,MEYGKQLTSHTTDIEGLLVFDFPVHGDNRGWFKENWQRTKMTNLGL...,NaN,5.1.3.-,2006-10-31,2004-11-23,2021-04-07,blast_match,-


In [58]:
cl_res.to_excel('/home/shizhenkun/codebase/DMLF/results/谷棒_results.xlsx', index=None)

In [45]:
cl_res[(cl_res.ec_ecrecer==cl_res.ec_deepec)& (cl_res.ec_ecrecer!='-')]

,id,id_uniprot,seq,seqlength,ec_ecrecer,date_integraged,date_sequence_update,date_annotation_update,res_type,ec_deepec
8,tr|Q8NQ78|Q8NQ78_CORGL,P63517,MSDFLNADGSLNVDKVREEFPILKRTVRDGKPLAYLDSGATSQRPE...,NaN,2.8.1.7,2004-10-11,2004-10-11,2021-09-29,blast_match,2.8.1.7
26,tr|Q8NP89|Q8NP89_CORGL,P43329,MTTSETAPSKASLYELLEGVSLSDERTFRRRLSKARAPKALGAIKA...,NaN,3.6.4.13,1995-11-01,2001-04-27,2021-09-29,blast_match,3.6.4.13
33,tr|Q8NRI1|Q8NRI1_CORGL,P77938,MTTHIDRIVGAALSEDAPWGDITSDTFIPGSAQLSAKVVAREPGVF...,NaN,2.4.2.19,1997-11-01,1997-02-01,2020-08-12,blast_match,2.4.2.19
36,tr|Q8NS39|Q8NS39_CORGL,P0A547,MIGAIWAQGRDGIIGDGTDMPWHIPEDLKHFKKTTMGQPVIMGRRT...,NaN,1.5.1.3,2005-03-15,2015-04-01,2021-04-07,blast_match,1.5.1.3
52,tr|Q8NQ63|Q8NQ63_CORGL,P0A585,MVIFGVTGDLARKKLLPAIYDLANRGLLPPGFSLVGYGRREWSKED...,NaN,1.1.1.49,2005-03-15,2005-03-15,2021-04-07,blast_match,1.1.1.49
...,...,...,...,...,...,...,...,...,...,...
1274,tr|Q8NSC1|Q8NSC1_CORGL,A0QTK3,MLGSIFTASAVVMILLGLGMLTVFTQRLVDQKIDIASSEIDRARVI...,NaN,2.7.13.3,2013-02-06,2013-02-06,2021-06-02,blast_match,2.7.13.3
1297,tr|Q8NU71|Q8NU71_CORGL,Q9RC53,MSVGGSDWKNFKEVDIIRFATRILVIQVATVALVVAICTGIFAVLM...,NaN,2.7.13.3,2002-06-06,2000-05-01,2021-09-29,blast_match,2.7.13.3
1303,tr|Q8NLX9|Q8NLX9_CORGL,Q8GIX7,MTTAAPQEFTAAVVEKFGHDVTVKDIDLPKPGPHQALVKVLTSGIC...,NaN,1.1.1.1,2020-10-07,2003-03-01,2021-06-02,blast_match,1.1.1.1
1313,tr|Q53ZX1|Q53ZX1_CORGT,Q8NT73,MTHMTSSNTARSAEWFEKAQKLTPGGVNSPVRAFGSVGGQARFIEK...,437.0,5.4.3.8,2002-10-19,2002-10-19,2021-09-29,db_match,5.4.3.8
